<a href="https://colab.research.google.com/github/AliciaFalconCaro/PythonColabExamples/blob/main/MVARforSyntheticCPETdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##code to be updated based on how it was implemented for the challenge

import numpy as np
import pandas as pd
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests

# Example function to apply VAR model and calculate Granger causality as a matrix
def apply_var_and_granger_as_matrix(data, window_size, overlap, max_lag):
    num_vars = data.shape[1]  # Number of variables
    num_points = len(data)     # Length of time series

    # Step size for overlapping windows (50% overlap)
    step_size = int(window_size * (1 - overlap))

    results_list = []

    variables = data.columns.tolist()

    # Loop over the data with the specified window and overlap
    for start in range(0, num_points - window_size + 1, step_size):
        # Extract the windowed data
        end = start + window_size
        window_data = data.iloc[start:end][variables]


        # Fit a VAR model to the windowed data
        model = VAR(window_data)
        results = model.fit(maxlags=max_lag, ic='aic')

        # Initialize Granger causality matrix
        granger_matrix = np.zeros((num_vars, num_vars))

        # Compute Granger causality for each pair of variables
        for i, var1 in enumerate(variables):
            for j, var2 in enumerate(variables):
                if i != j:  # No need to check causality for the same variable
                    test_result = grangercausalitytests(window_data[[var1, var2]], max_lag)
                    # Collect p-value for the Granger causality test (use max_lag F-test result)
                    p_value = test_result[max_lag][0]['ssr_ftest'][1]
                    granger_matrix[i, j] = p_value

        # Store the matrix along with window time range
        results_list.append({
            'start_time': start,
            'end_time': start + window_size,
            'granger_matrix': granger_matrix
        })

    return results_list


In [ ]:
# Example synthetic dataset

# Parameters
n_time_points = 1000  # Number of time points per subject
n_subjects = 5        # Number of subjects
np.random.seed(42)

# Create an empty list to hold data for all subjects
all_subjects_data = []

for subject in range(1, n_subjects + 1):
    # Generate synthetic time series data for CPET variables (non-stationary with trends)
    vo2 = 20 + 0.05 * np.arange(n_time_points) + np.random.normal(0, 2, n_time_points)
    vco2 = 15 + 0.03 * np.arange(n_time_points) + np.random.normal(0, 1.5, n_time_points)
    heart_rate = 70 + 0.1 * np.arange(n_time_points) + np.random.normal(0, 5, n_time_points)

    # Create a DataFrame for the current subject
    subject_data = pd.DataFrame({
        'VO2': vo2,
        'VCO2': vco2,
        'HeartRate': heart_rate,
        'Subject_ID': subject  # Add a subject identifier
    })

    # Append the subject data to the list
    all_subjects_data.append(subject_data)

# Concatenate all subject data into one DataFrame
combined_data = pd.concat(all_subjects_data, ignore_index=True)

# Print the first few rows of the combined data
#print(combined_data.head())

combined_data.head()

,VO2,VCO2,HeartRate,Subject_ID
0,20.993428,17.099033,66.624109,1
1,19.773471,16.416951,69.377407,1
2,21.395377,15.149446,66.237900,1
3,23.196060,14.119595,68.760192,1
4,19.731693,16.167335,60.931927,1


In [ ]:
# if we have timestamps, we can convert them to datetime to avoid problems with MVAR and reset the index to the datetime
# Convert 'Timestamp' to datetime format (if you have a timestamp column)
#data['Timestamp'] = pd.to_datetime(data['Timestamp'])

# Set the 'Timestamp' as the index (if you have such a column)
#data.set_index('Timestamp', inplace=True)


normalized_data = combined_data.groupby('Subject_ID').apply(lambda x: (x[['VO2', 'VCO2', 'HeartRate']] - x[['VO2', 'VCO2', 'HeartRate']].mean()) / x[['VO2', 'VCO2', 'HeartRate']].std()).reset_index(drop=True)
# Reattach the Subject column
normalized_data['Subject_ID'] = combined_data['Subject_ID'].values
normalized_data.head()


,VO2,VCO2,HeartRate,Subject_ID
0,-1.640518,-1.484533,-1.815320,1
1,-1.723837,-1.562470,-1.721643,1
2,-1.613066,-1.707299,-1.828460,1
3,-1.490084,-1.824973,-1.742643,1
4,-1.726691,-1.590992,-2.008987,1


In [ ]:
# Parameters
window_size = 200  # Window size in seconds (assuming 1 second per data point)
overlap = 0.5      # 50% overlap
max_lag = 10        # Maximum lag for VAR/Granger Causality

# Apply VAR and Granger causality to each subject's data over sliding windows


Streaming output truncated to the last 5000 lines.
ssr based F test:         F=1.4812  , p=0.1582  , df_denom=172, df_num=9
ssr based chi2 test:   chi2=14.8038 , p=0.0965  , df=9
likelihood ratio test: chi2=14.2581 , p=0.1134  , df=9
parameter F test:         F=1.4812  , p=0.1582  , df_denom=172, df_num=9

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=1.1415  , p=0.3343  , df_denom=169, df_num=10
ssr based chi2 test:   chi2=12.8334 , p=0.2331  , df=10
likelihood ratio test: chi2=12.4185 , p=0.2580  , df=10
parameter F test:         F=1.1415  , p=0.3343  , df_denom=169, df_num=10

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=23.4946 , p=0.0000  , df_denom=196, df_num=1
ssr based chi2 test:   chi2=23.8542 , p=0.0000  , df=1
likelihood ratio test: chi2=22.5294 , p=0.0000  , df=1
parameter F test:         F=23.4946 , p=0.0000  , df_denom=196, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=7.9161  , 

The GC p-value indicates the statistical significance of the test results. A low p-value (typically below 0.05 or 0.01) suggests that the null hypothesis (that X does not Granger cause Y) can be rejected, meaning X significantly helps predict Y.
If the p-value is large, we do not have enough evidence to say that X Granger causes Y.

# Plotting the GC values for easier visualization and interpretation

There are several ways to visually represent Granger Causality (GC) results from your DataFrame containing subject_id, phase_id, and the GC values per window. The best approach depends on what aspects of the results you want to emphasize (e.g., causality strength over time, between variables, or across subjects).
Examples based on saving the GC values, the person_id and the phase_id in a dataframe. GC values per pair saved as a separate feature per row.

**Heatmaps (for GC values between variables)**

A heatmap is useful for showing the strength of Granger causality between variables over different windows for each subject. You can create a heatmap per subject and window, with color intensity representing the GC p-values between each pair of variables.

X-axis: The target variables (e.g., Var1, Var2, Var3, etc.).
Y-axis: The source variables (e.g., Var1, Var2, Var3, etc.).
Color intensity: The GC values (p-values or strength).

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt

# Example GC values for a subject
subject_gc_values = gc_results_df.loc[gc_results_df['subject_id'] == 'subject1']

# Create a heatmap for a specific window
for index, row in subject_gc_values.iterrows():
    window_gc_values = row[4:].values.reshape((num_vars, num_vars))  # Reshape to original matrix size
    sns.heatmap(window_gc_values, annot=True, cmap='coolwarm', xticklabels=variables, yticklabels=variables)
    plt.title(f'Granger Causality for Subject {row["subject_id"]}, Phase {row["phase_id"]}, Window {index}')
    plt.show()




**Line Plot of GC Values Over Time (per variable pair)**

A line plot shows how the strength of causality between variable pairs evolves over time (or windows). This visualization is effective when you want to see trends in GC strength across different windows.

X-axis: Window time or window number.
Y-axis: GC values (p-values or strength).
Multiple lines: Each line represents the GC values for a specific variable pair (e.g., Var1->Var2, Var2->Var3).

In [ ]:
import matplotlib.pyplot as plt

# Extract GC values for one subject
subject_gc_values = gc_results_df.loc[gc_results_df['subject_id'] == 'subject1']

# Loop through each pair of variables and plot their GC values over windows
for var_i in variables:
    for var_j in variables:
        if var_i != var_j:
            gc_series = subject_gc_values[f'{var_i}->{var_j}_pvalue']
            plt.plot(subject_gc_values['WindowStart'], gc_series, label=f'{var_i}->{var_j}')

plt.title('Granger Causality Over Time for Subject 1')
plt.xlabel('Window Start')
plt.ylabel('GC P-Value')
plt.legend()
plt.show()


**Correlation Heatmaps Between Variables (over GC values)**

If you want to see how variables are related based on Granger causality across windows, you can compute the correlations of the GC values between variables and represent them in a correlation heatmap.

In [ ]:
correlation_matrix = gc_results_df.iloc[:, 4:].corr()  # Compute correlation between GC values
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation of GC P-Values Between Variables')
plt.show()


**Box Plots (for comparing GC values across subjects)**

Box plots can be used to compare the distribution of GC values across different subjects or phases for a specific variable pair. This is useful for spotting outliers and understanding the spread of GC values across subjects.

X-axis: Subject or phase.
Y-axis: GC values.
Boxes: Show the median, quartiles, and outliers.
Example (using seaborn):

In [ ]:
import seaborn as sns

# Box plot for one variable pair across subjects
sns.boxplot(x='subject_id', y='Var1->Var2_pvalue', data=gc_results_df)
plt.title('Distribution of GC P-Values (Var1->Var2) Across Subjects')
plt.show()


**3D Surface Plot (for windowed GC values across multiple variable pairs)**

A 3D surface plot can represent the change in GC values over time (windows) for each variable pair. This plot is useful when you want to explore GC strength in three dimensions: window, source variable, and target variable.

X-axis: Variable pairs.
Y-axis: Window number.
Z-axis: GC values.


In [ ]:
#the most representative maybe together with plotbox. We could plot it per Phase on average over patients with condition and without condition
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Example data
X = np.arange(len(gc_results_df))  # Window indices
Y = np.arange(num_pairs)  # Variable pairs
X, Y = np.meshgrid(X, Y)
Z = gc_results_df.iloc[:, 4:].values.T  # GC values for all windows and pairs

ax.plot_surface(X, Y, Z, cmap='viridis')
ax.set_xlabel('Window')
ax.set_ylabel('Variable Pair')
ax.set_zlabel('GC P-Value')
plt.title('3D Surface of GC P-Values Over Time')
plt.show()


# ML or DL exploration


We have the dataframe with the person_id, phase_id and GC pair-wise values. Separately, we also have another dataframe with some categorical data for each subject, such as gender or age range, and binary data for class A for each subject that represents if each subject has a disease or not. Use these two dataframes and a KNN model to predict Class A (if the subject has a disease).



In [ ]:
#confirm standarization for GC is done per subject (it may also be done per phase)

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import make_pipeline
import numpy as np

# Example DataFrames (replace with actual data)
gc_df = pd.DataFrame({
    'person_id': ['S1', 'S2', 'S3', 'S4', 'S5'],
    'phase_id': ['rest', 'activity', 'rest', 'activity', 'rest'],
    'Var1->Var2_pvalue': [0.01, 0.05, 0.02, 0.06, 0.03],
    'Var2->Var3_pvalue': [0.03, 0.07, 0.01, 0.08, 0.04],
    'Var1->Var3_pvalue': [0.02, 0.06, 0.03, 0.09, 0.05]
})

subject_info_df = pd.DataFrame({
    'person_id': ['S1', 'S2', 'S3', 'S4', 'S5'],
    'gender': ['M', 'F', 'M', 'F', 'M'],
    'age_range': ['20-30', '30-40', '20-30', '40-50', '30-40'],
    'Class A': [1, 0, 1, 0, 1]  # Binary label: 1 for disease, 0 for no disease
})

# Step 1: Merge the two DataFrames on person_id
merged_df = pd.merge(gc_df, subject_info_df, on='person_id')

# Step 2: One-hot encode categorical variables (gender, age_range, phase_id)
categorical_cols = ['gender', 'age_range', 'phase_id']
encoder = OneHotEncoder()
encoded_cats = encoder.fit_transform(merged_df[categorical_cols]).toarray()
encoded_cat_df = pd.DataFrame(encoded_cats, columns=encoder.get_feature_names_out(categorical_cols))

# Step 3: Prepare the final dataset by concatenating GC values and encoded categorical data
X = pd.concat([merged_df.drop(columns=categorical_cols + ['Class A', 'person_id']), encoded_cat_df], axis=1)
y = merged_df['Class A']  # Target variable
subject_ids = merged_df['person_id']  # Grouping by person_id for subject-wise splitting

# Step 4: Create the StratifiedGroupKFold object for cross-validation
skf = StratifiedGroupKFold(n_splits=5)

# Initialize lists to store performance metrics
accuracy_list = []
conf_matrix_list = []

# Step 5: Perform Stratified Group K-Fold Cross-Validation
for train_index, test_index in skf.split(X, y, groups=subject_ids):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Standardize the feature data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Step 6: Train the KNN model
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X_train_scaled, y_train)

    # Step 7: Make predictions
    y_pred = knn.predict(X_test_scaled)

    # Step 8: Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Store performance metrics
    accuracy_list.append(accuracy)
    conf_matrix_list.append(conf_matrix)

# Output cross-validation results
print(f"Average Accuracy: {np.mean(accuracy_list):.2f}")
print("Confusion Matrices for Each Fold:")
for i, cm in enumerate(conf_matrix_list):
    print(f"Fold {i + 1} Confusion Matrix:")
    print(cm)


In [ ]:
#if we were to use a simple 1D CNN:

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

# Example DataFrames (replace with actual data)
gc_df = pd.DataFrame({
    'person_id': ['S1', 'S2', 'S3', 'S4', 'S5'],
    'phase_id': ['rest', 'activity', 'rest', 'activity', 'rest'],
    'Var1->Var2_pvalue': [0.01, 0.05, 0.02, 0.06, 0.03],
    'Var2->Var3_pvalue': [0.03, 0.07, 0.01, 0.08, 0.04],
    'Var1->Var3_pvalue': [0.02, 0.06, 0.03, 0.09, 0.05]
})

subject_info_df = pd.DataFrame({
    'person_id': ['S1', 'S2', 'S3', 'S4', 'S5'],
    'gender': ['M', 'F', 'M', 'F', 'M'],
    'age_range': ['20-30', '30-40', '20-30', '40-50', '30-40'],
    'Class A': [1, 0, 1, 0, 1]  # Binary label: 1 for disease, 0 for no disease
})

# Step 1: Merge the two DataFrames on person_id
merged_df = pd.merge(gc_df, subject_info_df, on='person_id')

# Step 2: One-hot encode categorical variables (gender, age_range, phase_id)
categorical_cols = ['gender', 'age_range', 'phase_id']
encoder = OneHotEncoder()
encoded_cats = encoder.fit_transform(merged_df[categorical_cols]).toarray()
encoded_cat_df = pd.DataFrame(encoded_cats, columns=encoder.get_feature_names_out(categorical_cols))

# Step 3: Prepare the final dataset by concatenating GC values and encoded categorical data
X = pd.concat([merged_df.drop(columns=categorical_cols + ['Class A', 'person_id']), encoded_cat_df], axis=1)
y = merged_df['Class A']  # Target variable
subject_ids = merged_df['person_id']  # Grouping by person_id for subject-wise splitting

# Convert to numpy arrays for easier manipulation
X = X.values
y = y.values

# Step 4: Create the StratifiedGroupKFold object for cross-validation
skf = StratifiedGroupKFold(n_splits=5)

# Initialize lists to store performance metrics
accuracy_list = []
conf_matrix_list = []

# Step 5: Define a simple CNN model
def create_cnn(input_shape):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification output
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Step 6: Perform Stratified Group K-Fold Cross-Validation
for train_index, test_index in skf.split(X, y, groups=subject_ids):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Standardize the feature data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Reshape the data to be 3D for Conv1D input (samples, timesteps, features)
    X_train_scaled = np.expand_dims(X_train_scaled, axis=2)
    X_test_scaled = np.expand_dims(X_test_scaled, axis=2)

    # Step 7: Create and train the CNN model
    model = create_cnn(input_shape=(X_train_scaled.shape[1], 1))
    model.fit(X_train_scaled, y_train, epochs=10, batch_size=8, verbose=0)

    # Step 8: Make predictions
    y_pred = (model.predict(X_test_scaled) > 0.5).astype("int32").flatten()

    # Step 9: Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Store performance metrics
    accuracy_list.append(accuracy)
    conf_matrix_list.append(conf_matrix)

# Output cross-validation results
print(f"Average Accuracy: {np.mean(accuracy_list):.2f}")
print("Confusion Matrices for Each Fold:")
for i, cm in enumerate(conf_matrix_list):
    print(f"Fold {i + 1} Confusion Matrix:")
    print(cm)


In [ ]:
#to save any dataframe to a .csv file
df.to_csv('gc_results.csv', index=False)